In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
def load_netcdf_files(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.nc')]
    datasets = [xr.open_dataset(os.path.join(directory, f)) for f in files]
    return datasets


In [3]:
def merge_per_cycle(directory):
    datasets = load_netcdf_files(directory)
    
    merged = None

    for ds in datasets:
        if merged is None:
            merged = ds
        elif len(ds.iOBS) > 0:
            merged = xr.concat([merged, ds], dim='iOBS')

    return merged

In [4]:
directory9 = '/nobackup/sreich/swot/swot_obsfit_L3/cycle_009_llc270_30'
directory10 = '/nobackup/sreich/swot/swot_obsfit_L3/cycle_010_llc270_30'
#datasets = load_netcdf_files(directory)
#datasets

In [5]:
merged9 = merge_per_cycle(directory9)


In [6]:
merged10 = merge_per_cycle(directory10)

In [6]:
#ds_all = xr.concat([merged9, merged10], dim='iOBS')
ds_all = merged9
ds_all

<xarray.Dataset>
Dimensions:               (iOBS: 1606513)
Coordinates:
    longitude             (iOBS) float64 -75.06 -74.79 -73.69 ... -14.48 -14.77
    latitude              (iOBS) float64 -25.51 -24.83 -24.16 ... 155.1 155.1
Dimensions without coordinates: iOBS
Data variables: (12/15)
    obs_date              (iOBS) float64 7.393e+05 7.393e+05 ... 7.393e+05
    obs_YYYYMMDD          (iOBS) int64 20240106 20240106 ... 20240113 20240113
    obs_HHMMSS            (iOBS) int64 15511 15516 15531 ... 160054 160049
    sample_x              (iOBS) float64 -75.06 -74.79 -73.69 ... -14.48 -14.77
    sample_y              (iOBS) float64 -25.51 -24.83 -24.16 ... 155.1 155.1
    sample_z              (iOBS) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                    ...
    sample_interp_XC11    (iOBS) float64 -28.1 -28.1 -27.89 ... 172.2 172.2
    sample_interp_YC11    (iOBS) float64 -78.19 -78.19 -74.24 ... 56.91 56.91
    sample_interp_XCNINJ  (iOBS) float64 -18.03 -18.03 -18.17 ... -178.2 -178.2
    sample_interp_YCNINJ  (iOBS) float64 -74.36 -74.36 -70.07 ... 51.52 51.52
    sample_interp_i       (iOBS) float64 8.0 10.0 12.0 12.0 ... 29.0 29.0 29.0
    sample_interp_j       (iOBS) float64 25.0 27.0 5.0 6.0 ... 1.0 2.0 3.0 4.0

In [7]:
half = range(0, int(10000))
ds_all = ds_all.isel(iOBS=half)

In [8]:
obs = xr.Dataset(
    data_vars=dict(
        obs_date           =(["iOBS"], ds_all.obs_date.values),
        obs_YYYYMMDD       =(["iOBS"], ds_all.obs_YYYYMMDD.values),
        obs_HHMMSS         =(["iOBS"], ds_all.obs_HHMMSS.values), 
        sample_x           =(["iSAMPLE"], ds_all.sample_x.values),
        sample_y           =(["iSAMPLE"], ds_all.sample_y.values),
        sample_z           =(["iSAMPLE"], ds_all.sample_z.values),
        sample_type        =(["iSAMPLE"], ds_all.sample_type.values),
        obs_val            =(["iOBS"], ds_all.obs_val.values),
        obs_uncert         =(["iOBS"], np.ones(len(ds_all.sample_interp_i))*0.02),
        sample_interp_XC11 =(["iOBS"], ds_all.sample_interp_XC11.values ),
        sample_interp_YC11 =(["iOBS"], ds_all.sample_interp_YC11.values ),
        sample_interp_XCNINJ =(["iOBS"], ds_all.sample_interp_XCNINJ.values ),
        sample_interp_YCNINJ =(["iOBS"], ds_all.sample_interp_YCNINJ.values ),
        sample_interp_i =(["iOBS"], ds_all.sample_interp_i.values ),
        sample_interp_j =(["iOBS"], ds_all.sample_interp_j.values ),
        sample_interp_w =(["iOBS", "iINTERP"], np.ones((len(ds_all.sample_interp_i),8))/8  )
    ),
)

In [9]:
obs

<xarray.Dataset>
Dimensions:               (iOBS: 10000, iSAMPLE: 10000, iINTERP: 8)
Dimensions without coordinates: iOBS, iSAMPLE, iINTERP
Data variables: (12/16)
    obs_date              (iOBS) float64 7.393e+05 7.393e+05 ... 7.393e+05
    obs_YYYYMMDD          (iOBS) int64 20240106 20240106 ... 20240116 20240116
    obs_HHMMSS            (iOBS) int64 15511 15516 15531 ... 185245 185249
    sample_x              (iSAMPLE) float64 -75.06 -74.79 ... -40.3 -40.54
    sample_y              (iSAMPLE) float64 -25.51 -24.83 ... -61.83 -61.83
    sample_z              (iSAMPLE) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                    ...
    sample_interp_YC11    (iOBS) float64 -78.19 -78.19 -74.24 ... -62.06 -62.06
    sample_interp_XCNINJ  (iOBS) float64 -18.03 -18.03 -18.17 ... -58.17 -58.17
    sample_interp_YCNINJ  (iOBS) float64 -74.36 -74.36 -70.07 ... -66.2 -66.2
    sample_interp_i       (iOBS) float64 8.0 10.0 12.0 12.0 ... 26.0 26.0 26.0
    sample_interp_j       (iOBS) float64 25.0 27.0 5.0 6.0 ... 7.0 8.0 9.0 10.0
    sample_interp_w       (iOBS, iINTERP) float64 0.125 0.125 ... 0.125 0.125

In [10]:
data_dir = '/nobackup/sreich/swot/swot_obsfit_L3/'
fname =  'swot_cycles_009_llc270_30_test_obsfit.nc'
obs.to_netcdf(data_dir + fname)